## Import các thư viện

In [1]:
import matplotlib.pyplot as plt
from keras_nlp.api.models import DebertaV3Preprocessor
from keras_nlp.api.models import DebertaV3Classifier
import keras_nlp
import keras
import tensorflow as tf

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split

2024-05-27 06:01:33.261915: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 06:01:33.264549: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 06:01:33.302511: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 06:01:34.036048: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Tham khảo: https://www.kaggle.com/code/awsaf49/aes-2-0-kerasnlp-starter#%F0%9F%8D%BD%EF%B8%8F-%7C-Preprocessing

## Các tham số

In [2]:
# TODO: đổi path file csv
data_path = "train.csv"
# Sử dụng model DeBERTa-V3-Extra-Small-English
model_preset = "deberta_v3_extra_small_en"
max_epochs = 10
batch_size = 8
shufle = 1000
lr = 0.0000003

## Các hàm chính để xử lý model

### 1. Tiền xử lý dữ liệu

#### 1.1 Tiền xử lý label

 Label giá trị trị từ 1-6 (cột score) là dữ liệu categorical.
 Nhưng để làm bài này, label phải là dữ liệu ordinary
 Biến đổi label CATEGORICAL SANG ORDINARY bằng cách :

 Encode label thành một vector nhị phân có độ dài 6, bit i bằng 1 nghĩa là đã thỏa yêu cầu i
 Các giá trị trong vector CHO BIẾT XÁC SUẤT VỊ TRÍ i THỎA YÊU CẦU i.
 Từ đó có thể sử dụng CROSS-ENTROPY để sử dụng hàm lỗi so sánh sự tương đồng giữa 2 phân phối.

 Từ trên ta tạo tính ORDINARY như sau:

 Bài được chấm điểm 1 <= i <= 6 nghĩa là đã thỏa mãn yêu cầu i, VÀ CŨNG THỎA TẤT CẢ CÁC YÊU CẦU TRƯỚC ĐÓ (<= i),
 nên các bit trong vector nhãn sẽ = 1 ở các vị trí <= i, và 0 ở các vị trí còn lại.
 Cách làm này giúp có TÍNH SO SÁNH, vì một khi được điểm j >= i, thì các bit 1 của điểm i cũng = 1 ở trong j


In [3]:
def label_process(y):
    n = len(y)
    print()
    z = np.zeros((n, 6), "float32")
    for i in range(n):
        s = y[i]
        z[i, :s] = 1
    return z

#### 1.2 Tiền xử lý đầu vào

Lý do chúng ta cần thực hiện tiền xử lý đầu vào này là vì mô hình  DeBERTa không lấy văn bản thô làm đầu vào. Thay vào đó, chúng làm việc với cách biểu diễn kiểu số cho văn bản. DebertaV3Preprocessor xử lý các bước cần thiết để chuyển đổi văn bản thô thành các biểu diễn số này, bao gồm chuyển sang kiểu token (chia văn bản thành các từ hoặc từ phụ riêng lẻ) và mã hóa các token này dưới dạng số nguyên. Nó cũng đảm bảo rằng tất cả các chuỗi đầu vào đều có cùng độ dài 512, bằng cách đệm các chuỗi ngắn hơn hoặc cắt bớt các chuỗi dài hơn.

In [4]:
preprocessor = DebertaV3Preprocessor.from_preset(
    preset=model_preset,
    sequence_length=512,
)

def input_process(input, label):
    return (preprocessor(input), label)

### 2. Tạo và cấu hình model

Load model đã được cấu hình và train sẵn, sau đó train để fine-tune model này phù hợp cho bài toán. </br>
Như đã nói trên sử đầù ra là một vector độ dài 6, mỗi vị trí cho biết xác suất thỏa yêu cầu i.</br>
Đầu ra của mạng sử dụng sigmoid cho ra giá trị 0->1, chuẩn hóa thành xác suất cho mỗi vị trí i.</br>
Sử dụng hàm Binary Cross-Entropy, vì ta tính xác suất mỗi vị trí (mỗi vị trí đúng (1) đến sai (0)),
sau đó keras.losses.BinaryCrossentropy tổng hợp Loss tại mỗi vị trí làm Loss chung
Sử dụng Adam optimizer kết hợp SGD và momentum

In [5]:
def create_DebertaV3_model(model_preset):
    debertaV3 = DebertaV3Classifier.from_preset(
        model_preset, preprocessor=None, num_classes=6
    )

    #NOTE: Bug ở dòng sau 
    # inputs = debertaV3.input
    # outputs = debertaV3(inputs)

    # prob_outputs = keras.layers.Activation("sigmoid")(outputs)
    # model = keras.Model(inputs, prob_outputs)

    return debertaV3

### 3. Tạo dataset cho dữ liệu train và validation

Load dữ liệu từ file csv và tạo tf.data.Dataset cho dữ liệu train và dữ liệu validation.</br>
Dataset load dữ liệu theo lô (và cache để tiết kiệm bộ nhớ trong) và hoán vị dữ liệu sau mỗi vòng lặp cho THUẬT TOÁN SGD

In [6]:
def create_dataset():
    df = pd.read_csv(data_path)
    X_train_df, X_val_df, y_train_df, y_val_df = train_test_split(
        df["full_text"], df["score"], test_size=0.2, stratify=df["score"]
    )
    X_train = X_train_df.tolist()
    X_val = X_val_df.tolist()
    y_train = label_process(y_train_df.tolist())
    y_val = label_process(y_val_df.tolist())
    train_ds = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .map(input_process)
        .cache()
        .shuffle(shufle, seed=0)
        .batch(batch_size)
    )
    opt = tf.data.Options()
    opt.experimental_deterministic = False
    train_ds = train_ds.with_options(opt)

    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val))
        .map(input_process)
        .cache()
        .batch(batch_size)
    )
    print(train_ds)
    print(val_ds)
    return train_ds, val_ds

## Huấn luyện model

In [7]:
train_ds, val_ds = create_dataset()
model = create_DebertaV3_model(model_preset)

model.compile(
    optimizer=keras.optimizers.Adam(lr),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="accuracy")],
    jit_compile=True,
)
save_model = keras.callbacks.ModelCheckpoint(
    "model.weights.h5",  # HDF5
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="min",
)
# Train model
history = model.fit(
    x = train_ds,
    epochs=max_epochs,
    validation_data=val_ds,
    callbacks=[save_model],
)

<_OptionsDataset element_spec=({'token_ids': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None), 'padding_mask': TensorSpec(shape=(None, 512), dtype=tf.bool, name=None)}, TensorSpec(shape=(None, 6), dtype=tf.float32, name=None))>
<_BatchDataset element_spec=({'token_ids': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None), 'padding_mask': TensorSpec(shape=(None, 512), dtype=tf.bool, name=None)}, TensorSpec(shape=(None, 6), dtype=tf.float32, name=None))>
Epoch 1/10


I0000 00:00:1716764531.291348   14996 service.cc:145] XLA service 0x73b53c027600 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716764531.291393   14996 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-05-27 06:02:13.013097: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-27 06:02:14.736972: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at xla_ops.cc:580 : INVALID_ARGUMENT: Incompatible shapes: [8,6] vs. [8]
	 [[{{node Equal}}]]
	tf2xla conversion failed while converting __inference_one_step_on_data_60932[]. Run with TF_DUMP_GRAPH_PREFIX=/path/to/dump/dir and --vmodule=xla_compiler=2 to obtain a dump of the compiled functions.
2024-05-27 06:02:14.737059: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatib

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL InvalidArgumentError REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/decade/.local/lib/python3.10/site-packages/tensorflow/python/framework/errors_impl.py(284): __init__
  /home/decade/.local/lib/python3.10/site-packages/tensorflow/python/eager/execute.py(53): quick_execute
  /home/decade/.local/lib/python3.10/site-packages/tensorflow/python/eager/context.py(1500): call_function
  /home/decade/.local/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py(251): call_flat
  /home/decade/.local/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py(216): call_preflattened
  /home/decade/.local/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/concrete_function.py(1322): _call_flat
  /home/decade/.local/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_fu

## Thống kê lỗi và đánh giá

In [ ]:
training_loss = history.history["loss"]
validation_loss = history.history["val_loss"]

epoch_rows = range(1, len(training_loss) + 1)
plt.plot(epoch_rows, training_loss, "r--")
plt.plot(epoch_rows, validation_loss, "b-")
plt.legend(["Training Loss", "Validation Loss"])
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.savefig("Loss_plot.png")
plt.show()